In [137]:
import pandas as pd
import numpy as np
import os
import math

In [138]:
os.chdir('/Users/arvind/Documents/Work/Zoho WorkDrive (Educational Initiatives Private Limited)/My Folders/Projects/SALT/Dashboards/')

In [139]:
df = pd.read_excel("./SLASAssessmentDataWithDistricts.xlsx")

In [140]:
var_list = list(df)

In [141]:
unwanted = ('student_id','disecode','Districts','SM','student_name','class','section','roll_no','unique_id',
            'gender','social_category','medium_id','English_paperset','English_jumble_set','English_present_absent',
             'EnglishTotal','EnglishPercentage_old','EnglishPercentage','English_cheating_drop','Language_paperset',
             'Language_jumble_set','Language_present_absent','LanguageTotal','LanguagePercentage_old',
             'LanguagePercentage','Language_cheating_drop','Maths_paperset','Maths_jumble_set','Maths_present_absent',
             'MathsTotal','MathsPercentage_old','MathsPercentage','Maths_cheating_drop')

In [142]:
retain = [ele for ele in var_list if ele not in unwanted]

In [143]:
df_unpivot = pd.melt(df,id_vars = ['student_id','disecode','Districts'
                                         ,'student_name','class','section','roll_no'
                                         ,'unique_id','gender','social_category'
                                         ,'English_present_absent','Language_present_absent','Maths_present_absent']
                        ,value_vars = retain)

In [144]:
df_unpivot[['S','Q']] = df_unpivot['variable'].str.split('_',1,expand=True)

In [145]:
df_unpivot[['Subject','Question']] = df_unpivot['Q'].str.split('Q',1,expand=True)

In [146]:
df_unpivot.loc[df_unpivot["Subject"]=="N","Subject"]="English"
df_unpivot.loc[df_unpivot["Subject"]=="L","Subject"]="Language"
df_unpivot.loc[df_unpivot["Subject"]=="M","Subject"]="Maths"

In [147]:
df_unpivot = df_unpivot.drop(['section','roll_no','unique_id','variable','S','Q'],axis=1)

In [148]:
df_unpivot.fillna('',inplace = True)

In [149]:
indexELM = df_unpivot[(
                          ((df_unpivot['English_present_absent']=='A')&(df_unpivot['Subject']=='English'))|
                          ((df_unpivot['Language_present_absent']=='A')&(df_unpivot['Subject']=='Language'))|
                          ((df_unpivot['Maths_present_absent']=='A')&(df_unpivot['Subject']=='Maths'))|
                          ((df_unpivot['English_present_absent']=='')&(df_unpivot['Subject']=='English'))|
                          ((df_unpivot['Language_present_absent']=='')&(df_unpivot['Subject']=='Language'))|
                          ((df_unpivot['Maths_present_absent']=='')&(df_unpivot['Subject']=='Maths'))
                        )
                         ].index
df_unpivot.drop(indexELM, inplace=True)

In [150]:
df_unpivot['Question'] = df_unpivot['Question'].astype(np.int64)

In [151]:
mapping = pd.read_csv("./287_questionwise_optionwise_setwise_performance_after_cheating_modified.csv")

In [152]:
mapping = mapping.rename({'Class':'class'},axis=1)
mapping = mapping.rename({'Qno':'Question'},axis=1)

In [153]:
dft = pd.merge(df_unpivot,mapping,how='left',on=['class','Subject','Question'])

In [154]:
dft = dft.drop(['English_present_absent','Language_present_absent','Maths_present_absent'],axis=1)

In [155]:
dft.to_csv("./slas_data_for_dashboard.csv")